# Algoritmos genéticos
**Guillermo Segura Gómez**

Los Algoritmos Genéticos son técnicas de optimización inspiradas en la teoría de la evolución de Charles Darwin. Utilizan conceptos de selección natural y genética para encontrar soluciones óptimas a problemas complejos. Los GA operan mediante una población de individuos que representan posibles soluciones. Cada individuo se evalúa utilizando una función de fitness, y los individuos con mayor aptitud tienen una mayor probabilidad de reproducirse y pasar sus características a la siguiente generación. Los principales operadores en un GA son la selección, el cruce (crossover) y la mutación.

### Componentes de un algoritmo genético

1. **Población Inicial**: Un conjunto de posibles soluciones.
2. **Función de Fitness**: Evalúa la calidad de cada solución.
3. **Selección**: Escoge las mejores soluciones para reproducirse.
4. **Crossover**: Combina dos soluciones padres para crear una nueva solución.
5. **Mutación**: Introduce variaciones aleatorias en las soluciones.
6. **Iteración**: Repite el proceso de selección, cruce y mutación para mejorar las soluciones a lo largo de varias generaciones.

## Deap

La librería *deap* (Distributed Evolutionary Algorithms in Python) es una librería implementada en python para poder emplear de manera sencilla los algoritmos genéticos. Deap posee características únicas, como por ejemplo, la posibilidad de crear inicializadores, abiertos y personalizados. Además es posible escoger el operador fitness que se quiera según el problema. 

### Types

Primero es necesario definir el tipo de problema que se busca resolver. Deap permite definir esto utilizando el módulo *creator*. Creamos una clase para un problema de minimización *FitnessMin*, además una clase *Individual* que es derivada de una lista con atributo de fitness. 

In [9]:
from deap import creator, base
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

/Users/guillermo_sego/anaconda3/envs/PLN/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/guillermo_sego/anaconda3/envs/PLN/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


### Inicialización

2. **base.Toolbox**:
   - Es una caja de herramientas que almacena los operadores y funciones necesarias para el GA.
   - **`toolbox.register`**: Registra funciones y operadores personalizados en la caja de herramientas.

In [4]:
from deap import base
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -5, 5)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

3. **tools**:
   - Proporciona operadores genéticos predefinidos, como selección, cruce y mutación.
   - **`tools.initRepeat`**: Inicializa una población repitiendo una función de generación de individuos.
   - **`tools.cxBlend`**: Operador de cruce que combina dos individuos.
   - **`tools.mutGaussian`**: Operador de mutación que añade ruido gaussiano.
   - **`tools.selTournament`**: Operador de selección por torneo.

In [5]:
from deap import tools
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

4. **algorithms**:
   - Proporciona implementaciones de algoritmos genéticos estándar.
   - **`algorithms.eaSimple`**: Implementa un algoritmo genético simple.

In [1]:
# Librerias
import random
import numpy as np
from deap import base, creator, tools, algorithms

Configuración de parámetros e inicialización

In [7]:
# Definir la función de fitness y minimizarla
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Crear el toolbox y configurar los parámetros del GA
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -5, 5)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=2)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Definir la función objetivo (función de esfera)
def evalSphere(individual):
    return sum(x**2 for x in individual),

toolbox.register("evaluate", evalSphere)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

# Configuración de parámetros del GA
population = toolbox.population(n=50)
probab_crossover = 0.7
probab_mutate = 0.2
num_generations = 40

/Users/guillermo_sego/anaconda3/envs/PLN/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/Users/guillermo_sego/anaconda3/envs/PLN/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [8]:
# Algoritmo Genético
if __name__ == "__main__":
    print("Start of evolution")
    
    # Evaluar la población inicial
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    print("  Evaluated %i individuals" % len(population))

    # Bucle evolutivo
    for gen in range(num_generations):
        print("-- Generation %i --" % gen)

        # Selección
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        # Aplicar cruce y mutación
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < probab_crossover:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < probab_mutate:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluar individuos con fitness no calculado
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        print("  Evaluated %i individuals" % len(invalid_ind))

        # Reemplazar la población con la nueva generación
        population[:] = offspring

        # Recopilar estadísticas
        fits = [ind.fitness.values[0] for ind in population]

        length = len(population)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5

        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)

    print("-- End of (successful) evolution --")

    best_ind = tools.selBest(population, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

Start of evolution
  Evaluated 50 individuals
-- Generation 0 --
  Evaluated 28 individuals
  Min 0.7137853230076491
  Max 29.80383151183243
  Avg 10.52156244131568
  Std 6.075860632692147
-- Generation 1 --
  Evaluated 29 individuals
  Min 0.07763914787365568
  Max 10.798139904057585
  Avg 3.4463993102352504
  Std 3.308904676221234
-- Generation 2 --
  Evaluated 37 individuals
  Min 0.07763914787365568
  Max 9.962823979662643
  Avg 1.5766730502781419
  Std 1.9344604565585832
-- Generation 3 --
  Evaluated 34 individuals
  Min 0.03957257966396463
  Max 7.859001771075116
  Avg 0.7523777302435012
  Std 1.3120467581880684
-- Generation 4 --
  Evaluated 38 individuals
  Min 0.03957257966396463
  Max 5.566030423764439
  Avg 0.6428280830478853
  Std 1.0087389070678188
-- Generation 5 --
  Evaluated 33 individuals
  Min 0.02733619355068219
  Max 0.7137853230076491
  Avg 0.1643689359328623
  Std 0.16189455452083185
-- Generation 6 --
  Evaluated 37 individuals
  Min 0.011289919811300515
  Max 